#CIS Week 10 Homework

**Instructor:** Lyle Ungar

**Content Creators:** Sanjeevini Ganni, Brittany Shields, Alessandra Rossi

In [1]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = '' #@param {type:"string"}
my_pod = 'Select' #@param ['Select', 'upain', 'ah-damn-optimizer', 'backpropagandists', 'backpropers','excel-erators','GAN-gsters','han-not-solo','hufflefluffs','lets-taco-bout-it','natural-networkers','pytorture','sigmoids','strong-signals','the-denominators','the-travellers', 'the-weekenders', 'tomorrows-incredibles', 'brute-force']
my_email = '' #@param {type:"string"}

# start timing
import time
try:t0;
except NameError: t0 = time.time()

Go to Runtime -> Change runtime type -> Set Hardware Accelerator to "GPU"

## Part I: Finetune GPT-2





GPT2 is a large-scale transformer-based language model developed by OpenAI. It is pre-trained on a large corpus of dataset of 8 million web pages has around 1.5 billion parameters. GPT2 is useful for language generation tasks, that is it predicts the next word, given some text.  

In this excersice we will be finetuning DistillGPT-2, distilled version of GPT2. We want the language model to generate wikipedia style of text. Hence, we will finetune distillGPT on [wikitext-2-raw-v1](https://huggingface.co/datasets/wikitext) dataset scrapped from wikipedia articles.

In [2]:
#@title Install
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 6.5 MB 31.8 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 134 kB 58.6 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 1.1 MB 55.5 MB/s 
     |████████████████████████████████| 127 kB 52.6 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 50.9 MB/s 
     |████████████████████████████████| 271 kB 70.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [3]:
#@title Imports
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

In [4]:
#@title Load Datset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Example of Dataset:

In [5]:
datasets["train"][48]

{'text': " Two manga adaptations were produced , following each of the game 's main female protagonists Imca and Riela . They were Senjō no Valkyria 3 : Namo naki Chikai no Hana ( 戦場のヴァルキュリア3 名もなき誓いの花 , lit . Valkyria of the Battlefield 3 : The Flower of the Nameless Oath ) , illustrated by Naoyuki Fujisawa and eventually released in two volumes after being serialized in Dengeki Maoh between 2011 and 2012 ; and Senjō no Valkyria 3 : -Akaki Unmei no Ikusa Otome- ( 戦場のヴァルキュリア3 -赤き運命の戦乙女- , lit . Valkyria of the Battlefield 3 -The Valkyrie of the Crimson Fate ) , illustrated by Mizuki Tsuge and eventually released in a single volume by Kadokawa Shoten in 2012 . \n"}

####Tokenizer

In [6]:
#Define the model Checkpoint
model_checkpoint = "distilgpt2"

#Tokenizer  
#To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. 
#This is all done by the AutoTokenizer class:  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#We define a function to call the tokenizer on our texts:
def tokenize_function(examples):
    return tokenizer(examples["text"])

#Apply tokenizer to all the splits in our datasets object and drop the text coolumn
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
#we need to concatenate all our texts together then split the result in small chunks of a certain block_size
#Define the block size for training
block_size = 128

#preprocessing function that will group our texts
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/10 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/10 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

###Load pre-trained model

In [8]:
#####Pretrained model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/336M [00:00<?, ?B/s]

###Run Generations before Finetuning

In [9]:
def run_generations(PROMPT_TEXT):
  device = model.device
  input_ids = tokenizer.encode(PROMPT_TEXT, return_tensors="pt").to(device)

  sample_output = model.generate(
      input_ids, 
      do_sample=True, 
      max_length=100, 
      top_p=0.95,
      num_return_sequences=3, 
      early_stopping=True
  )


  for i, output in enumerate(sample_output):
    print("Generation: "+ str(i) +"\n" + 100 * '-')
    print(tokenizer.decode(output, skip_special_tokens=True))
    print()


########
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generation: 0
----------------------------------------------------------------------------------------------------
Fast and the Furious   the new franchise is coming out in North America, starting with a few other releases in the future.


In fact, both of these are being released early on, so if you‡re looking to see this new franchise that‪d be your go-to movie of all time, they will probably be coming out in March, at around the same time as we did the previous ones.
You can see the latest in trailers below, if you

Generation: 1
----------------------------------------------------------------------------------------------------
Fast and the Furious   will become the biggest film of all time. With its stunning score, the first movie can be viewed at www.moviesuniverse.com. For more information about the film, visit www.moviesuniverse.com. For more information about the film, visit www.moviesuniverse.com.

Generation: 2
----------------------------------------------------------------

In [10]:
############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = ...
run_generations(PROMPT_TEXT)


PROMPT_TEXT = ...
run_generations(PROMPT_TEXT)

TypeError: ignored

###Fine-Tuning (This will take around 20-25 minutes)

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18666
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4668


Epoch,Training Loss,Validation Loss


Perplexity of Validation Set

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

###Run Generations after Finetuning
Run generations for different prompts and compare them with the initial generations.

In [ ]:
PROMPT_TEXT = 'Fast and the Furious '
run_generations(PROMPT_TEXT)

############## Run Generation for different prompt texts of your choice
PROMPT_TEXT = ...
run_generations(PROMPT_TEXT)


PROMPT_TEXT = ...
run_generations(PROMPT_TEXT)

## Question 1
How do the generations after finetuning compare to the generations before finetuning? Do you think finetuning achieved its purpose of generating Wikipedia like texts? (max. 200 words)


In [ ]:
q1 = "x" #@param{type:"string"}

try:t0;
except NameError: t0 = time.time()

## Part II: Primer on Gender Bias in Word Embeddings

Read the following primer that describes the societal implications of gender bias in word embeddings: \\
“Man is to Doctor as Woman is to Nurse: The Gender Bias of Word Embeddings: Why we should worry about gender inequality in Natural Language Processing techniques” \\
https://towardsdatascience.com/gender-bias-word-embeddings-76d9806a0e17 \\
*9 min read*, Tommaso Buonocore, 2019


## Question 2

Identify a compelling quote or example from the article that resonated with you.  How does it emphasize the societal responsibility of data scientists? (max. 200 words)

In [ ]:
q2 = "x" #@param{type:"string"}

try:t1;
except NameError: t1 = time.time()

##Part III: Tech Paper on Gender Bias in Word Embeddings

In the previous article, the author referenced that data scientists are introducing new strategies to reduce biases in word embeddings.  In 2016, researchers from Boston University and Microsoft Research published a paper with significant findings demonstrating the prevalence of gender bias in word embeddings, as well as proposed mitigation measures.  A 5-minute summary of the technical components of the paper can be found [here](https://towardsdatascience.com/tackling-gender-bias-in-word-embeddings-c965f4076a10) and the original paper can be found [here](https://arxiv.org/abs/1607.06520). \

“Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings” \
https://arxiv.org/abs/1607.06520 \
*30 min read*, 2016





## Question 3 
In the “Discussion” section, the authors consider the responsibility of machine language programmers with regards to bias in word embeddings.  Describe their stance. (max. 200 words)

In [ ]:
q3 = "x" #@param{type:"string"}

## Question 4

How could word embeddings amplify societal stereotypes? How could the authors’ proposed technological interventions address this concern? (max. 300 words)

In [ ]:
q4 = "x" #@param{type:"string"}

##Part IV: Fake News

The following article provides an overview of neural fake news, including potential defenses against it as well as suggestions for further research.

“An Exhaustive Guide to Detecting and Fighting Neural Fake News using NLP” \
•	https://www.analyticsvidhya.com/blog/2019/12/detect-fight-neural-fake-news-nlp/ \
•	*16-minute read*, Analytics Vidhya, 2019

In 2019, Jack Clark, the Policy Director at OpenAI, gave testimony to the US House Intelligence Committee.  You can either read his written 10-page testimony here or watch his live testimony here (jump to minute 11:00 to watch his 5 minute testimony).

Written Testimony of Jack Clark, Policy Director, OpenAI \
•	https://intelligence.house.gov/uploadedfiles/clark_deepfakes_sfr.pdf \
•	or \
•	https://www.c-span.org/video/?461679-1/house-intelligence-committee-hearing-deepfake-videos \
*Jump to minute 11:00* \
•	House Permanent Select Committee on Intelligence, 2019 \


## Question 5

Jack Clark enumerated several ideas for interventions.  Discuss one of the specific ideas he suggested. (max. 300 words)

In [ ]:
q5 = "x" #@param{type:"string"}

## PART V: Know your Pod

You know the drill ;-) Ask two pod members with which two individuals they'd share a meal, living or dead, if they could, and why. (max. 200 words)

In [ ]:
know_your_pod = "x" #@param{type:"string"}

#Submission

Once you're done, click on 'Share' and add the link to the box below.

In [ ]:
#@markdown #Run Cell to Show Airtable Form
#@markdown Confirm your answers and then click "Submit". If you're having any issues, fill out the form here: https://airtable.com/shrcl81YacGQCUesG.

import time
import numpy as np
import urllib.parse
from IPython.display import IFrame
def prefill_form(src, fields: dict):
  '''
  src: the original src url to embed the form
  fields: a dictionary of field:value pairs,
  e.g. {"pennkey": my_pennkey, "location": my_location}
  '''
  prefill_fields = {}
  for key in fields:
      new_key = 'prefill_' + key
      prefill_fields[new_key] = fields[key]
  prefills = urllib.parse.urlencode(prefill_fields)
  src = src + prefills
  return src


#autofill time if it is not present
try: t0;
except NameError: t0 = time.time()
try: t1;
except NameError: t1 = time.time()

# autofill fields if they are not present
# a missing pennkey and pod will result in an Airtable warning
# which is easily fixed user-side.
try: my_pennkey;
except NameError: my_pennkey = ""
try: my_pod;
except NameError: my_pod = "Select"
try: my_email;
except NameError: my_email = ""
try: q1;
except NameError: q1 = ""
try: q2;
except NameError: q2 = ""
try: q3;
except NameError: q3 = ""
try: q4;
except NameError: q4 = ""
try: q5;
except NameError: q5 = ""
try: know_your_pod;
except NameError: know_your_pod = ""


times = np.array([t1])-t0

fields = {
    "pennkey": my_pennkey,
    "pod": my_pod, 
    "email": my_email, 
    "q1": q1,
    "q2": q2,
    "q3": q3,
    "q4": q4,
    "q5": q5,
    "know_your_pod": know_your_pod,
    "cumulative_times": times
}

src = "https://airtable.com/embed/shrcl81YacGQCUesG?"

display(IFrame(src = prefill_form(src, fields), width = 800, height = 400))